In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Conv3D, MaxPool3D
from keras.optimizers import Adam,SGD
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

%matplotlib inline
train_path = './data/Train/'
test_path = './data/Test/'
img_size = 45
batch_size = 32
#177x254

train_datagen = ImageDataGenerator(rescale = 1. /255, preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(rescale = 1. /255, preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_path, target_size=(177, 254), batch_size=32)
train_generator.class_indices

test_generator = test_datagen.flow_from_directory(test_path, target_size=(177, 254), batch_size=32)
test_generator.class_indices

In [ ]:
n_batch = math.ceil(train_generator.samples / batch_size)
n_batch

In [ ]:
random_img = np.random.choice(train_generator.filenames)
img = Image.open(train_path + random_img)
print(train_path+random_img)
print(img.size)
img

In [ ]:
batch_x, batch_y = train_generator.next()
batch_x.shape, batch_y.shape

In [ ]:
from keras.layers import BatchNormalization, Dropout

ygo_model = Sequential()

ygo_model.add(Conv2D(10,(5,5), padding = "same", activation='sigmoid', input_shape=(177,254,3)))
ygo_model.add(BatchNormalization())
ygo_model.add(MaxPool2D())

ygo_model.add(Dropout(0.3))
ygo_model.add(Conv2D(8,(8,8), padding="same", activation='relu'))
ygo_model.add(MaxPool2D())
ygo_model.add(BatchNormalization())

ygo_model.add(Flatten())
ygo_model.add(Dropout(0.2))
ygo_model.add(Dense(10, activation='softmax'))

ygo_model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(1e-5),
                      metrics=['accuracy'])
ygo_model.summary()

In [ ]:
log = ygo_model.fit_generator(train_generator, steps_per_epoch=n_batch, epochs = 50, validation_data = test_generator, validation_steps = len(test_generator))

In [ ]:
#from jupyterthemes import jtplot
#jtplot.style(spines=False,grid=True)

plt.plot(log.history['val_loss'], label='validation_set',color='#51c44f')
plt.plot(log.history['loss'], label='train_set',color='#4e5ac7')
plt.legend()

In [ ]:
lbl_dict = {k:i for i,k in train_generator.class_indices.items()}
lbl_dict

In [ ]:
def get_class(path):
    img = Image.open(path).convert('RGB')
    imgn = img.resize((254,177))
    plt.imshow(img)
    plt.axis('off')
    x = np.asarray(imgn, np.float32)[None]
    x = preprocess_input(x)
    y = ygo_model.predict(x)
    
    preds = lbl_dict[np.argmax(y)]
    return preds

In [ ]:
random_path = './data/Test/' + np.random.choice(test_generator.filenames)
print(random_path)
get_class(random_path)

In [ ]:
from keras.models import Model
from keras.preprocessing import image

layer_outputs = [layer.output for layer in ygo_model.layers[:100]]
activation_model = Model(inputs=ygo_model.input, outputs=layer_outputs)
test_image = random_path
img = image.load_img(test_image, target_size=(177, 254))
img_arr = image.img_to_array(img)
img_arr = np.expand_dims(img_arr, axis=0)
img_arr /= 255.

In [ ]:
#jtplot.style(spines=False,grid=False)

activations = activation_model.predict(img_arr)
activations[1].shape

In [ ]:
plt.matshow(activations[0][0, :, :, 1], cmap='seismic')
plt.show()

In [ ]:
plt.matshow(activations[1][0, :, :, 1], cmap='seismic')
plt.show()

In [ ]:
plt.matshow(activations[2][0, :, :, 1], cmap='seismic')
plt.show()

In [ ]:
plt.matshow(activations[3][0, :, :, 1], cmap='seismic')
plt.show()

In [ ]:
plt.matshow(activations[4][0, :, :, 2], cmap='seismic')
plt.show()

In [ ]:
plt.matshow(activations[5][0, :, :, 2], cmap='seismic')
plt.show()

In [ ]:
plt.matshow(activations[6][0, :, :, 2], cmap='seismic')
plt.show()

In [ ]:
import keras.backend as K
kernels = ygo_model.layers[4].kernel
kernels = K.eval(kernels)
kernels.shape

In [ ]:
kernels = kernels[:,:,0,:]
kernels.shape

In [ ]:
fig, axes = plt.subplots(1, kernels.shape[2], figsize=(16,4))
for i in range(kernels.shape[2]):
    axes[i].imshow(kernels[:,:,i], 'gray')
    axes[i].axis('off')